<a href="https://colab.research.google.com/github/sofi-jo/SistemaReconoLenguSenVA/blob/main/Entrenamiento_Dataset_LenguajeSe%C3%B1as.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Antes de correr el código debemos tener el entorno en GPU

# **Instalación de paquetes**

In [ ]:
%%capture
!pip install super-gradients==3.1.0
!pip install imutils
!pip install roboflow
!pip install pytube --upgrade

**super-gradients** es una biblioteca de entrenamiento de código abierto para modelos de aprendizaje profundo basados en PyTorch. Permite entrenar o ajustar modelos de visión por computadora de última generación

**imutils** es una biblioteca de Python que proporciona una serie de funciones útiles para trabajar con imágenes, como redimensionar, rotar y recortar imágenes

**roboflow** es una plataforma de entrenamiento de modelos de aprendizaje profundo que proporciona herramientas para etiquetar y preparar conjuntos de datos de imágenes para el entrenamiento de modelos

**pytube** es una biblioteca de Python que permite descargar videos de YouTube y extraer información de ellos


Reiniciar tiempo de ejecución

# **Importación de las librerías requeridas**

 SuperGradients es una biblioteca de entrenamiento de código abierto para modelos de aprendizaje profundo basados en PyTorch. Permite entrenar o ajustar modelos de visión por computadora de última generación.
La clase Trainer se utiliza para crear un objeto de entrenamiento para entrenar modelos. Tiene varios parámetros que se pueden especificar, como el nombre del experimento y los parámetros de entrenamiento. El módulo dataloaders proporciona funciones para cargar conjuntos de datos en el formato YOLO para entrenamiento y validación.

In [ ]:
from super_gradients.training import Trainer
from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val

In [ ]:
from IPython.display import clear_output

In [ ]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback


In [ ]:
from super_gradients.training import models


# Check Point e Inicialización de entrenador

In [ ]:
CHECKPOINT_DIR = 'checkpoints2'
trainer = Trainer(experiment_name='sign_yolonas_run2', ckpt_root_dir=CHECKPOINT_DIR)

# **Exportar Dataset Roboflow a Google Colab**

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="IuYv6KOKs5p62rFSLvGa")
project = rf.workspace("david-lee-d0rhs").project("american-sign-language-letters")
dataset = project.version(6).download("yolov5")

# **Parametros de dataset en un diccionario**

In [ ]:
dataset_params = {
    'data_dir':'/content/American-Sign-Language-Letters-6',
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'test_images_dir':'test/images',
    'test_labels_dir':'test/labels',
    'classes': ['A', 'B', 'C', 'D', 'E', 'F', 'G','H', 'I', 'J', 'K', 'L','M','N','O', 'P', 'Q', 'R', 'S', 'T', 'U',  'V', 'W', 'X', 'Y', 'Z']
}

Parametros a argumentos

In [ ]:
train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

clear_output()

# **Inspeccionar data set definido previamente**

In [ ]:
train_data.dataset.transforms

In [ ]:
train_data.dataset.dataset_params['transforms'][1]

In [ ]:
train_data.dataset.dataset_params['transforms'][1]['DetectionRandomAffine']['degrees'] = 10.42

In [ ]:
train_data.dataset.plot()

# **Iniciar el modelo**

Una vez iniciado el modelo con lo previo. A continuación iniciamos el modelo para finetuning. Se debe agregar el num_classes que iniciamos previamente aquí

In [ ]:
model = models.get('yolo_nas_s',
                   num_classes=len(dataset_params['classes']),
                   pretrained_weights="coco"
                   )

# **Definir metricas y parametros de entramiento**

Se debe definir los paramentros de entrenamiento para que el entrenamiento pueda empezar

## **Existen paramentros obligatorios que se deben definir para los paramentro de entrenamiento**



*   max_epochs - El el numero maximo de epoch de entrenamiento
*   loss - la cantidad de perdida que vamos a usar
*   optimizer - Optimizer que se usara
*   train_metrics_list - metricas que se cargan durante del entrenamiento
*   valid_metrics_list - metricas que se cargan durante del entrenamiento
*   metric_to_watch - metricas en donde se guardara el checkpoint





In [ ]:
train_params = {
    # ENABLING SILENT MODE
    'silent_mode': True,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    # ONLY TRAINING FOR 10 EPOCHS FOR THIS EXAMPLE NOTEBOOK
    "max_epochs": 25,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        # NOTE: num_classes needs to be defined here
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            # NOTE: num_classes needs to be defined here
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

# **Videos Demo para entrenamiento**

In [ ]:
!gdown "https://drive.google.com/uc?id=1bLlfh9gr6JxE9Wacw4xgv5FNijA7j9GZ&confirm=t"

In [ ]:
!gdown "https://drive.google.com/uc?id=1kju9OOo05c5XTfrKuLvsZcxu_NPloQk_&confirm=t"

# **Entrenamiento del Modelo**

In [ ]:
trainer.train(model=model,
              training_params=train_params,
              train_loader=train_data,
              valid_loader=val_data)

In [ ]:
best_model = models.get('yolo_nas_s',
                        num_classes=len(dataset_params['classes']),
                        checkpoint_path="checkpoints2/sign_yolonas_run2/ckpt_best.pth")